# Wind Speed and Direction

HRRR returns grib files with estimates at specific locations in a projected grid across the spherical surface of the earth. Becasue of this projection, the `u` and `v` components for wind speed and other values are not directly E or N respectively. By taking a second order estimate at the closest points to our location of interest, we estimate locally how much u and v contribute locally to the E and W vectors.

In [8]:
import sys
sys.path.append('..')
print(sys.path)
from gnomy import constants, core, utils

from herbie import Herbie

['../gnomy/', '../gnomy/', '../gnomy/', '../gnomy/', '../gnomy/', 'C:\\Users\\liama\\.config\\herbie', 'c:\\Users\\liama\\OneDrive\\NewVSCode\\gnomy\\notebooks', 'C:\\Program Files\\PerkinElmerInformatics\\ChemOffice2020\\ChemScript\\Lib', 'c:\\Users\\liama\\miniconda3\\envs\\lonestar\\python311.zip', 'c:\\Users\\liama\\miniconda3\\envs\\lonestar\\DLLs', 'c:\\Users\\liama\\miniconda3\\envs\\lonestar\\Lib', 'c:\\Users\\liama\\miniconda3\\envs\\lonestar', '', 'c:\\Users\\liama\\miniconda3\\envs\\lonestar\\Lib\\site-packages', 'c:\\Users\\liama\\miniconda3\\envs\\lonestar\\Lib\\site-packages\\win32', 'c:\\Users\\liama\\miniconda3\\envs\\lonestar\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\liama\\miniconda3\\envs\\lonestar\\Lib\\site-packages\\Pythonwin', '..', '..', '..']


get wind speeds from HRRR analysis with Herbie

In [9]:
date="2023-01-01 12:00"
latitude=29.25
longitude=360-98.31

# get data
H = Herbie(date, model='hrrr', product='sfc', fxx=0)
ds = H.xarray("(:UGRD:10 m above ground:anl)|(:VGRD:10 m above ground:anl)")
dsi = ds.herbie.nearest_points((longitude, latitude))
u10 = dsi.u10.values[0]
v10 = dsi.v10.values[0]
print(f"Wind speed at nearest point:\n\tu: {u10:.2f} m/s\n\tv: {v10:.2f} m/s")

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2023-Jan-01 12:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


c:\Users\liama\miniconda3\envs\lonestar\Lib\site-packages\herbie\core.py:797: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  logic = df.search_this.str.contains(searchString)


👨🏻‍🏭 Created directory: [C:\Users\liama\data\hrrr\20230101]


c:\Users\liama\miniconda3\envs\lonestar\Lib\site-packages\herbie\accessors.py:260: UserWarning: More than one time coordinate present for variable  "gribfile_projection".
  ds = ds.metpy.parse_cf()
c:\Users\liama\miniconda3\envs\lonestar\Lib\site-packages\herbie\accessors.py:264: UserWarning: More than one time coordinate present for variable  "u10".
  ds = ds.metpy.assign_y_x()
c:\Users\liama\miniconda3\envs\lonestar\Lib\site-packages\herbie\accessors.py:264: UserWarning: More than one time coordinate present for variable  "v10".
  ds = ds.metpy.assign_y_x()
c:\Users\liama\miniconda3\envs\lonestar\Lib\site-packages\herbie\accessors.py:264: UserWarning: More than one time coordinate present for variable  "gribfile_projection".
  ds = ds.metpy.assign_y_x()


Wind speed at nearest point:
	u: 0.14 m/s
	v: 1.24 m/s


c:\Users\liama\miniconda3\envs\lonestar\Lib\site-packages\pyproj\transformer.py:820: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return self._transformer._transform_point(


Get u and v components in the longitude and latitude directions.  
$$V_{\text{wind speed E/W}} = u10 * uc[0] + v10 * vc[0]$$
$$V_{\text{wind speed N/S}} = u10 * uc[1] + v10 * vc[1]$$

In [10]:
uc,vc = utils.get_coordinate_projections(date,latitude,longitude)
print(f"U components: {uc} m/s (lon, lat)")
print(f"V components: {vc} m/s (lon, lat)")
Ve = u10*uc[0] + v10*vc[0]
Vn = u10*uc[1] + v10*vc[1]
print(f"Wind speed at nearest point:\n\tV East: {Ve:.2f} m/s\n\tV North: {Vn:.2f} m/s")

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2023-Jan-01 12:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


c:\Users\liama\miniconda3\envs\lonestar\Lib\site-packages\herbie\core.py:797: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  logic = df.search_this.str.contains(searchString)


U components: (0.9999712998197906, 0.007576248195411632) m/s (lon, lat)
V components: (-0.009952709059404449, 0.999950470564607) m/s (lon, lat)
Wind speed at nearest point:
	V East: 0.13 m/s
	V North: 1.24 m/s


Finally, get the compass direction and total wind speed

In [11]:
compass_heading = utils.get_wind_direction(Ve,Vn)
print(f"Wind direction at nearest point: {compass_heading:.0f} degrees")

total_wind_speed = utils.get_wind_speed(Ve,Vn)
print(f"Total wind speed at nearest point: {total_wind_speed:.2f} m/s")

Wind direction at nearest point: 6 degrees
Total wind speed at nearest point: 1.25 m/s
